In [83]:
# count dna_reads in a file
# open file 
path_to_file = '/Users/dariapodorskaja/Desktop/Lab/Lane_7_NoIndex_L007_R1_001.fastq'
dna_reads = []
n = 160000 # number of lines in a file
#Result = []

# make lists of dna reads
with open(path_to_file, 'r') as f:   # f = open(path_to_file, 'r') # f is a variable with a file
    for line_number in range(n):
        if line_number %4 == 1:
            s = f.readline()
            s = s.replace('\n', '')
            dna_reads.append(s)
        else:
            next(f)

In [143]:
def disrupted_poly_A(read,min_len_poly_a=8, min_len_poly_a2 = 5):
    """
    find poly A tail in a read that is disrupted only by one base other than 'A'
    
    """
    import re
    min_len_poly_a = str(min_len_poly_a)
    min_len_poly_a2 = str(min_len_poly_a2)
    pattern = "[A]{" + min_len_poly_a + ",}[^A][A]{" + min_len_poly_a2 + ",}"
    m = re.search(pattern, read)  
    if m:
        return m.group()
    return ''

In [144]:
assert disrupted_poly_A("AATAA", 1, 1) == 'AATAA'
assert disrupted_poly_A("CCCAATAACCC", 1, 1) == 'AATAA'
assert disrupted_poly_A("CCGTGTACTATT", 2, 2) == ''
assert disrupted_poly_A("AAATAAAGAAA", 1, 1) == 'AAATAAA'
assert disrupted_poly_A("AAATAAAGAAA", 4, 4) == ''

In [142]:
for read in dna_reads:
    result = disrupted_poly_A(read, 7, 5)
    if result:
        print(result)

16
20
35
30
18
30
17
17
50
21
20
33
20
20
20
20
30
16
44
16
14
27
49
23
17
19
26
20
18
20
14
20
42
15
18
17
17
43
16
20
20
27
40
17
49
21
37
13
15
17
43
24
24
27
18
24
17
43
16
38
44
14
39
34
14
43
19
16
19
17
25
22
41
24
27
29
25
17
43
27
25
18
29
15
51
19
24
22
23
28
47
15
18
28
22
14
32
29
21
21
24
19
30
22
15
26
41
15
47
26
15
20
43
15
28
24
17
40
14
32
51
15
41
50
33
15
43
15
23
48
20
14
16
20
34
43
15
17
17
16
52
51
20
51
20
52
19
28
18
44
39
18
17
15
20
48
32
50
37
36
12
17
16
42
20
49
22
26
31
32
51
15
14
32
37
31
33
14
28
31
19
20
17
20
27
35
23
47
37
40
30
30
17
45
29
21
19
37
14
16
25
17
20
14
23
23
15
53
25
22
16
20
36
30
17
20
48
21
27
49
31
21
21
19
14
24
37
27
16
19
41
28
24
21
31
48
45
20
16
37
26
27
33
18
21
25
31
49
54
18
45
41
20
37
35
23
24
20
27
20
23
20
15
29
39
42
45
35
26
35
17
15
26
51
37
50
21
28
36
49
43
51
21
20
41
31
45
34
43
28
13
23
15
21
49
48
14
24
23
20
41
30
37
27
40
14
17
15
17
36
15
16
18
21
43
13
24
33
27
27
16
34
13
26
24
20
39
14
27
33
14
27
14
1

In [11]:
s = 'AAAT'
s = s[0:-1]

In [12]:
s

'AAA'

In [134]:
import re
s = 'CCCAATAACCC'
length = str(1)
pattern = "[A]{" + length + ",}([^A])[A]{" + length + ",}"

print(pattern)
m = re.search(pattern, s)
print(m.group(1))
print(m.span(1))
print(s[5])

[A]{1,}([^A])[A]{1,}
T
(5, 6)
T


In [131]:
def base_calling_error_probabilities(ascii_code):
    assert isinstance(ascii_code, str)
    assert len(ascii_code) == 1
    q = ord(ascii_code) - 33
    p = round(10 ** (-q / 10), 5)
    return p

In [132]:
phred('@')

0.00079

In [ ]:
def disrupted_poly_A(read,min_len_poly_a=8, min_len_poly_a2 = 5):
    """
    find poly A tail in a read that is disrupted only by one base other than 'A'
    
    """
    import re
    min_len_poly_a = str(min_len_poly_a)
    min_len_poly_a2 = str(min_len_poly_a2)
    pattern = "[A]{" + min_len_poly_a + ",}([^A])[A]{" + min_len_poly_a2 + ",}"
    m = re.search(pattern, read)  
    if m:
        return m.span(1)[0]
    return ''